# 🔧 Feature Engineering - Prédiction Churn Orange

**Objectif** : Créer des features avancées pour améliorer la performance des modèles

## 🎯 Features à Créer

### 1. Variables RFM (Recency, Frequency, Monetary)
- `tenure_group` : Catégorisation ancienneté (Nouveau/Moyen/Fidèle)
- `monthly_to_total_ratio` : Ratio charges mensuelles/totales
- `revenue_per_tenure` : Revenu moyen par mois d'ancienneté

### 2. Variables Comportementales
- `total_services` : Nombre total de services actifs
- `has_premium_services` : Au moins un service premium (0/1)
- `has_multiple_lines` : Lignes multiples (0/1)

### 3. Variables Contractuelles
- `is_month_to_month` : Contrat mensuel (0/1)
- `has_paperless_billing` : Facturation électronique (0/1)
- `payment_method_encoded` : Encodage méthode paiement

### 4. Variables Démographiques
- `is_senior` : Senior citizen (0/1)
- `has_family` : Avec partenaire ou dépendants (0/1)
- `demographic_risk` : Score de risque démographique

### 5. Variables Financières
- `charges_category` : Catégorie de dépenses (Low/Medium/High)
- `total_charges_clean` : TotalCharges converti en numérique

---

**Dataset principal** : IBM Telco (7,043 clients)

In [1]:
# ============================================
# IMPORTS & CONFIGURATION
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# Style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Imports réussis")
print(f"📦 Pandas : {pd.__version__}")
print(f"📦 Numpy  : {np.__version__}")

✅ Imports réussis
📦 Pandas : 2.1.4
📦 Numpy  : 1.25.2


In [2]:
# ============================================
# CHARGEMENT DONNÉES
# ============================================

# Charger dataset principal
df = pd.read_csv("../donnees/brutes/telco_customer_churn_ibm.csv")

print("="*80)
print("📊 DATASET CHARGÉ")
print("="*80)
print(f"✅ Shape      : {df.shape}")
print(f"✅ Mémoire    : {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("="*80)

# Aperçu
print("\n👀 APERÇU:")
display(df.head(3))

# Créer copie pour feature engineering (garder original intact)
df_features = df.copy()

print("\n✅ Copie créée pour feature engineering")

📊 DATASET CHARGÉ
✅ Shape      : (7043, 21)
✅ Mémoire    : 7.79 MB

👀 APERÇU:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes



✅ Copie créée pour feature engineering


In [3]:
# ============================================
# NETTOYAGE : TotalCharges
# ============================================

print("="*80)
print("🔧 NETTOYAGE TOTALCHARGES")
print("="*80)

# TotalCharges est parfois en string avec espaces
print(f"\nType actuel : {df_features['TotalCharges'].dtype}")
print(f"Valeurs uniques : {df_features['TotalCharges'].nunique()}")

# Conversion en numérique (les espaces deviennent NaN)
df_features['TotalCharges'] = pd.to_numeric(
    df_features['TotalCharges'], 
    errors='coerce'
)

# Vérifier valeurs manquantes
missing = df_features['TotalCharges'].isnull().sum()
print(f"\n⚠️  Valeurs manquantes après conversion : {missing}")

if missing > 0:
    # Remplacer NaN par 0 (nouveaux clients)
    # Alternative : remplacer par MonthlyCharges (estimation)
    df_features['TotalCharges'].fillna(
        df_features['MonthlyCharges'], 
        inplace=True
    )
    print(f"✅ NaN remplacés par MonthlyCharges (nouveaux clients)")

print(f"\n✅ TotalCharges converti en : {df_features['TotalCharges'].dtype}")
print(f"✅ Min: {df_features['TotalCharges'].min():.2f}")
print(f"✅ Max: {df_features['TotalCharges'].max():.2f}")
print(f"✅ Moyenne: {df_features['TotalCharges'].mean():.2f}")
print("="*80)

🔧 NETTOYAGE TOTALCHARGES

Type actuel : object
Valeurs uniques : 6531

⚠️  Valeurs manquantes après conversion : 11
✅ NaN remplacés par MonthlyCharges (nouveaux clients)

✅ TotalCharges converti en : float64
✅ Min: 18.80
✅ Max: 8684.80
✅ Moyenne: 2279.80


In [4]:
# ============================================
# FEATURE ENGINEERING : VARIABLES RFM
# ============================================

print("\n" + "="*80)
print("🔨 CRÉATION FEATURES RFM")
print("="*80)

# 1. Tenure Groups (catégorisation ancienneté)
df_features['tenure_group'] = pd.cut(
    df_features['tenure'],
    bins=[0, 12, 24, 48, 72],
    labels=['0-12 mois', '12-24 mois', '24-48 mois', '48+ mois']
)

# 2. Monthly to Total Ratio
df_features['monthly_to_total_ratio'] = (
    df_features['MonthlyCharges'] / 
    (df_features['TotalCharges'] + 1)  # +1 pour éviter division par zéro
)

# 3. Revenue per Tenure Month
df_features['revenue_per_tenure'] = (
    df_features['TotalCharges'] / 
    (df_features['tenure'] + 1)  # +1 pour éviter division par zéro
)

# 4. Charges Category
df_features['charges_category'] = pd.cut(
    df_features['MonthlyCharges'],
    bins=[0, 35, 70, 120],
    labels=['Low', 'Medium', 'High']
)

print("\n✅ Features RFM créées:")
print(f"   • tenure_group            : {df_features['tenure_group'].nunique()} catégories")
print(f"   • monthly_to_total_ratio  : ratio continu")
print(f"   • revenue_per_tenure      : {df_features['revenue_per_tenure'].min():.2f} à {df_features['revenue_per_tenure'].max():.2f}")
print(f"   • charges_category        : {df_features['charges_category'].nunique()} catégories")

# Aperçu
print("\n📊 Aperçu Features RFM:")
display(df_features[['tenure', 'tenure_group', 'MonthlyCharges', 
                      'TotalCharges', 'monthly_to_total_ratio', 
                      'revenue_per_tenure', 'charges_category']].head())

print("="*80)


🔨 CRÉATION FEATURES RFM

✅ Features RFM créées:
   • tenure_group            : 4 catégories
   • monthly_to_total_ratio  : ratio continu
   • revenue_per_tenure      : 9.18 à 118.97
   • charges_category        : 3 catégories

📊 Aperçu Features RFM:


,tenure,tenure_group,MonthlyCharges,TotalCharges,monthly_to_total_ratio,revenue_per_tenure,charges_category
0,1,0-12 mois,29.85,29.85,0.967585,14.925000,Low
1,34,24-48 mois,56.95,1889.50,0.030124,53.985714,Medium
2,2,0-12 mois,53.85,108.15,0.493358,36.050000,Medium
3,45,24-48 mois,42.30,1840.75,0.022967,40.016304,Medium
4,2,0-12 mois,70.70,151.65,0.463151,50.550000,High


In [5]:
# ============================================
# FEATURE ENGINEERING : VARIABLES COMPORTEMENTALES
# ============================================

print("\n" + "="*80)
print("🔨 CRÉATION FEATURES COMPORTEMENTALES")
print("="*80)

# Liste des services
services = [
    'PhoneService', 'MultipleLines', 'InternetService',
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# 1. Total Services (compter "Yes")
df_features['total_services'] = 0
for service in services:
    if service in df_features.columns:
        df_features['total_services'] += (df_features[service] == 'Yes').astype(int)

# 2. Has Premium Services
premium_services = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport']
df_features['has_premium_services'] = 0
for service in premium_services:
    if service in df_features.columns:
        df_features['has_premium_services'] += (df_features[service] == 'Yes').astype(int)

df_features['has_premium_services'] = (df_features['has_premium_services'] > 0).astype(int)

# 3. Has Multiple Lines
df_features['has_multiple_lines'] = (df_features['MultipleLines'] == 'Yes').astype(int)

# 4. Has Internet Service
df_features['has_internet'] = (df_features['InternetService'] != 'No').astype(int)

# 5. Has Streaming Services
df_features['has_streaming'] = (
    ((df_features['StreamingTV'] == 'Yes') | (df_features['StreamingMovies'] == 'Yes'))
).astype(int)

print("\n✅ Features Comportementales créées:")
print(f"   • total_services         : 0 à {df_features['total_services'].max()} services")
print(f"   • has_premium_services   : {df_features['has_premium_services'].sum()} clients avec premium")
print(f"   • has_multiple_lines     : {df_features['has_multiple_lines'].sum()} clients")
print(f"   • has_internet           : {df_features['has_internet'].sum()} clients")
print(f"   • has_streaming          : {df_features['has_streaming'].sum()} clients")

# Distribution
print("\n📊 Distribution Total Services:")
print(df_features['total_services'].value_counts().sort_index())

print("="*80)


🔨 CRÉATION FEATURES COMPORTEMENTALES

✅ Features Comportementales créées:
   • total_services         : 0 à 8 services
   • has_premium_services   : 4250 clients avec premium
   • has_multiple_lines     : 2971 clients
   • has_internet           : 5517 clients
   • has_streaming          : 3499 clients

📊 Distribution Total Services:
total_services
0      80
1    1701
2    1188
3     965
4     922
5     908
6     676
7     395
8     208
Name: count, dtype: int64


In [6]:
# ============================================
# FEATURES CONTRACTUELLES & DÉMOGRAPHIQUES
# ============================================

print("\n" + "="*80)
print("🔨 CRÉATION FEATURES CONTRACTUELLES & DÉMOGRAPHIQUES")
print("="*80)

# CONTRACTUELLES
df_features['is_month_to_month'] = (df_features['Contract'] == 'Month-to-month').astype(int)
df_features['has_paperless_billing'] = (df_features['PaperlessBilling'] == 'Yes').astype(int)

# DÉMOGRAPHIQUES
df_features['is_senior'] = df_features['SeniorCitizen']  # Déjà en 0/1
df_features['has_partner'] = (df_features['Partner'] == 'Yes').astype(int)
df_features['has_dependents'] = (df_features['Dependents'] == 'Yes').astype(int)

# Variable composite : Has Family
df_features['has_family'] = (
    (df_features['has_partner'] == 1) | (df_features['has_dependents'] == 1)
).astype(int)

# Demographic Risk Score (plus de facteurs de risque = plus de risque churn)
df_features['demographic_risk'] = (
    df_features['is_senior'] + 
    (1 - df_features['has_family'])  # Pas de famille = risque
).astype(int)

print("\n✅ Features Contractuelles:")
print(f"   • is_month_to_month      : {df_features['is_month_to_month'].sum()} clients")
print(f"   • has_paperless_billing  : {df_features['has_paperless_billing'].sum()} clients")

print("\n✅ Features Démographiques:")
print(f"   • is_senior              : {df_features['is_senior'].sum()} clients")
print(f"   • has_partner            : {df_features['has_partner'].sum()} clients")
print(f"   • has_dependents         : {df_features['has_dependents'].sum()} clients")
print(f"   • has_family             : {df_features['has_family'].sum()} clients")
print(f"   • demographic_risk       : 0 à {df_features['demographic_risk'].max()}")

print("="*80)


🔨 CRÉATION FEATURES CONTRACTUELLES & DÉMOGRAPHIQUES

✅ Features Contractuelles:
   • is_month_to_month      : 3875 clients
   • has_paperless_billing  : 4171 clients

✅ Features Démographiques:
   • is_senior              : 1142 clients
   • has_partner            : 3402 clients
   • has_dependents         : 2110 clients
   • has_family             : 3763 clients
   • demographic_risk       : 0 à 2


In [11]:
# ============================================
# ENCODAGE VARIABLE CIBLE
# ============================================

print("\n" + "="*80)
print("🎯 ENCODAGE TARGET : CHURN")
print("="*80)

# Créer version binaire (0/1)
df_features['Churn_binary'] = (df_features['Churn'] == 'Yes').astype(int)

print(f"\n✅ Churn encodé:")
print(f"   • Original (string)  : {df_features['Churn'].dtype}")
print(f"   • Binaire (int)      : {df_features['Churn_binary'].dtype}")

print(f"\n📊 Distribution:")
print(df_features['Churn_binary'].value_counts())
print(f"\nTaux de churn : {df_features['Churn_binary'].mean()*100:.2f}%")

print("="*80)


🎯 ENCODAGE TARGET : CHURN

✅ Churn encodé:
   • Original (string)  : object
   • Binaire (int)      : int64

📊 Distribution:
Churn_binary
0    5174
1    1869
Name: count, dtype: int64

Taux de churn : 26.54%


In [13]:
# ============================================
# RÉSUMÉ FEATURES ENGINEERING
# ============================================

print("\n" + "="*80)
print("📊 RÉSUMÉ FEATURES CRÉÉES")
print("="*80)

# Colonnes originales vs nouvelles
cols_originales = df.columns.tolist()
cols_nouvelles = [col for col in df_features.columns if col not in cols_originales]

print(f"\n✅ Colonnes originales  : {len(cols_originales)}")
print(f"✅ Nouvelles features   : {len(cols_nouvelles)}")
print(f"✅ Total colonnes       : {len(df_features.columns)}")

print(f"\n📋 NOUVELLES FEATURES ({len(cols_nouvelles)}):")
for i, col in enumerate(cols_nouvelles, 1):
    dtype = str(df_features[col].dtype)  # ← CORRECTION ICI
    nunique = df_features[col].nunique()
    print(f"   {i:2d}. {col:30s} ({dtype:15s}) - {nunique:4d} valeurs uniques")

# Taille mémoire
print(f"\n💾 MÉMOIRE:")
print(f"   • Avant : {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   • Après : {df_features.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Aperçu des nouvelles features
print(f"\n👀 APERÇU NOUVELLES FEATURES:")
display(df_features[cols_nouvelles].head())

print("="*80)


📊 RÉSUMÉ FEATURES CRÉÉES

✅ Colonnes originales  : 21
✅ Nouvelles features   : 17
✅ Total colonnes       : 38

📋 NOUVELLES FEATURES (17):
    1. tenure_group                   (category       ) -    4 valeurs uniques
    2. monthly_to_total_ratio         (float64        ) - 6782 valeurs uniques
    3. revenue_per_tenure             (float64        ) - 6660 valeurs uniques
    4. charges_category               (category       ) -    3 valeurs uniques
    5. total_services                 (int64          ) -    9 valeurs uniques
    6. has_premium_services           (int64          ) -    2 valeurs uniques
    7. has_multiple_lines             (int64          ) -    2 valeurs uniques
    8. has_internet                   (int64          ) -    2 valeurs uniques
    9. has_streaming                  (int64          ) -    2 valeurs uniques
   10. is_month_to_month              (int64          ) -    2 valeurs uniques
   11. has_paperless_billing          (int64          ) -    2 valeurs 

,tenure_group,monthly_to_total_ratio,revenue_per_tenure,charges_category,total_services,has_premium_services,has_multiple_lines,has_internet,has_streaming,is_month_to_month,has_paperless_billing,is_senior,has_partner,has_dependents,has_family,demographic_risk,Churn_binary
0,0-12 mois,0.967585,14.925000,Low,1,1,0,1,0,1,1,0,1,0,1,0,0
1,24-48 mois,0.030124,53.985714,Medium,3,1,0,1,0,0,0,0,0,0,0,1,0
2,0-12 mois,0.493358,36.050000,Medium,3,1,0,1,0,1,1,0,0,0,0,1,1
3,24-48 mois,0.022967,40.016304,Medium,3,1,0,1,0,0,0,0,0,0,0,1,0
4,0-12 mois,0.463151,50.550000,High,1,0,0,1,0,1,1,0,0,0,0,1,1
